In [3]:
import geopandas as gpd
import shapely.geometry
import os

def create_square_from_points(input_file, output_file):
    # Read the input file (GeoJSON or shapefile)
    gdf = gpd.read_file(input_file)
        
    # Ensure that the GeoDataFrame contains only points
    if not all(gdf.geometry.geom_type == "Point"):
        raise ValueError("The input file does not contain only Point geometries.")
    
    # Reproject to a projected CRS (UTM or other suitable for the area)
    projected_gdf = gdf.to_crs(epsg=32633)  # Replace EPSG:32633 with an appropriate UTM zone for your data
    
    # Compute the centroid of all points
    centroid = projected_gdf.unary_union.centroid
        
    # Calculate the maximum distance from the centroid to any point
    max_distance = projected_gdf.geometry.distance(centroid).max()
        
    # Define the side length of the square (twice the max distance)
    side_length = 2 * max_distance
        
    # Create the square polygon
    square = shapely.geometry.box(
        centroid.x - side_length / 2, centroid.y - side_length / 2,
        centroid.x + side_length / 2, centroid.y + side_length / 2
    )
        
    # Create a new GeoDataFrame for the square
    square_gdf = gpd.GeoDataFrame({'geometry': [square]}, crs=projected_gdf.crs)
    
    # Reproject the square back to the original CRS (if needed)
    square_gdf = square_gdf.to_crs(gdf.crs)
        
    # Save the square as a shapefile
    square_gdf.to_file(output_file, driver='ESRI Shapefile')

if __name__ == "__main__":
    input_file = "~/Documents/GitHub/UWW200_Master_Thesis_public/SpectralPatang/test_cut_shp/14_Flux_Towers_Zona.shp"  # or "path/to/your/input.shp"
    output_file = "~/Documents/GitHub/UWW200_Master_Thesis_public/SpectralPatang/test_cut_shp/output_square_python.shp"
        
    create_square_from_points(input_file, output_file)
    print(f"Square shapefile created and saved to {output_file}")


Square shapefile created and saved to ~/Documents/GitHub/UWW200_Master_Thesis_public/SpectralPatang/test_cut_shp/output_square_python.shp
